# JSOC Emergency

Status:
All resources from the Stanford JSOC website and data system are unavailable until repairs and recovery are completed
IRIS Level 2 data through November 22, 2024 remain available at iris.lmsal.com/search/
The Virtual Solar Observatory is the best location to find historical AIA and HMI science data through November 26, 2024, though not all dates and products are available
SDO EVE data products are not impacted by this outage

Useful Python script for downloading SDO data from HelioCloud (courtesy APL):

- https://solarweb1.stanford.edu/assets/scripts/AIA_CloudCatalog_final.py

```
Author Johns Hopkins University Applied Physics Laboratory LLC, 2024
License: MIT License

How to use CloudCatalog for accessing AIA data from HelioCloud
see also https://pypi.org/project/cloudcatalog/
use 'pip install cloudcatalog' and 'pip install s3fs' to use
```

In [ ]:
import os
import cloudcatalog
import astropy.io.fits
import s3fs

In [ ]:
# Sample searches of the entire heliophysics cloud catalog
search = cloudcatalog.EntireCatalogSearch()

# Return all the AIA datasets
aia_datasets = search.search_by_id('aia')
print(f"Searching for any 'aia', there are {len(aia_datasets)} datasets")

Searching for any 'aia', there are 9 datasets


In [ ]:
[e['id'] for e in aia_datasets]

['aia_0094',
 'aia_0131',
 'aia_0171',
 'aia_0193',
 'aia_0211',
 'aia_0304',
 'aia_0335',
 'aia_1600',
 'aia_1700']

In [ ]:
# Return by keywords, in this case AIA channels
ids = [e['id'] for e in search.search_by_keywords(['193', '211', '304'])]

print("\nSearching for any datasets with '193', '211' or '304' in them, just the dataIDs",ids)
print("\nSearch for just '211', all metadata:",search.search_by_id('211'))


Searching for any datasets with '193', '211' or '304' in them, just the dataIDs ['aia_0193', 'aia_0211', 'aia_0304', 'euvml_304', 'euvml_soho_304', 'euvml_stereoa_304', 'euvml_stereob_304', 'THG_L2_MAG_DED']

Search for just '211', all metadata: [{'id': 'aia_0211', 'index': 's3://gov-nasa-hdrl-data1/sdo/aia/registries/', 'title': 'AIA 0211 FITS data', 'start': '2015-06-01T00:00:00Z', 'stop': '2021-12-31T23:59:59Z', 'modification': '2023-05-04:00:00:00Z', 'indextype': 'csv', 'filetype': 'fits'}]


In [ ]:
# Search for AIA 193 files in a time range
# Set up the particular ID and the start and stop times
myid, start, stop = 'aia_0193', '2020-02-01T00:00:00Z', '2020-02-02T00:00:00Z'

# Point the search to the NASA HDRL Heliocloud instance
fr = cloudcatalog.CloudCatalog("s3://gov-nasa-hdrl-data1/")

# Perform the search
mycat = fr.request_cloud_catalog(myid, start_date=start, stop_date=stop)
print(f"{len(mycat)} files found")

351 files found


In [ ]:
# Get the filepaths for the first three files
print("\nLooking at the filelist query result")
filelist = mycat['datakey'].to_list()
print(f"For dataID {myid} over times {start}-{stop}, the first 3 files found are:\n{filelist[0:3]}\n")


Looking at the filelist query result
For dataID aia_0193 over times 2020-02-01T00:00:00Z-2020-02-02T00:00:00Z, the first 3 files found are:
['s3://gov-nasa-hdrl-data1/sdo/aia/20200201/0193/sdo_aia_h2_20200201T000000_0193_v1.fits', 's3://gov-nasa-hdrl-data1/sdo/aia/20200201/0193/sdo_aia_h2_20200201T000400_0193_v1.fits', 's3://gov-nasa-hdrl-data1/sdo/aia/20200201/0193/sdo_aia_h2_20200201T000800_0193_v1.fits']



In [ ]:
# Run a simple calc on first 3 files
# For packages that not yet cloud-aware, need to connect to S3 on AWS
fs = s3fs.S3FileSystem(anon=True)
for i in range(3):
    myitem = mycat.iloc[i]
    datastart = myitem['start']
    filename = myitem['datakey']
    hdul = astropy.io.fits.open(fs.open(filename))
    fs.download(filename, os.path.expanduser('~'))
    print(datastart,", mean value=",hdul[1].data.mean())

2020-02-01 00:00:00 , mean value= 191.15523592
2020-02-01 00:04:00 , mean value= 191.21884312
2020-02-01 00:08:00 , mean value= 191.32419744
